# Graph Retriever Workflow

This notebook shows the minimal steps for indexing documents with `Ragdoll.ingest_with_graph()` and querying the resulting graph retriever alongside the vector store.

In [ ]:
from langchain_core.documents import Document
from ragdoll.pipeline import IngestionOptions
from ragdoll.ragdoll import Ragdoll
import asyncio

documents = [
    Document(
        page_content=(
            "Ada Lovelace collaborated with Charles Babbage on the Analytical Engine."
        ),
        metadata={"source": "notebook"},
    )
]

options = IngestionOptions(
    skip_vector_store=True,
    entity_extraction_options={
        "config": {
            "graph_retriever": {
                "enabled": True,
                "backend": "simple",
                "top_k": 5,
            }
        }
    },
)
ragdoll = Ragdoll()

async def run_pipeline():
    result = await ragdoll.ingest_with_graph(documents, options=options)
    print("Ingestion stats:", result["stats"])
    return result

result = asyncio.run(run_pipeline())

In [ ]:
retriever = result["graph_retriever"]
if retriever:
    answers = retriever.invoke("Who collaborated with Charles Babbage?")
    for doc in answers:
        print(doc.page_content, doc.metadata)
else:
    print("Graph retriever not available. Check entity_extraction.graph_retriever in config.")